## Training Dataset for Images Analysis


In [1]:
from google.colab import files
# Upload images from your local system
uploaded = files.upload()


Saving overview_slide_1_3_4114c1a31e.jpg to overview_slide_1_3_4114c1a31e.jpg
Saving overview_slide_1_5_e0695f611f.jpg to overview_slide_1_5_e0695f611f.jpg
Saving about_6_fd826b8b77.jpg to about_6_fd826b8b77.jpg
Saving Ahmedabad1_a47e274d4d.png to Ahmedabad1_a47e274d4d.png


In [2]:
import os

# Create a folder for images
os.makedirs("images", exist_ok=True)

# Move uploaded files to the images directory
for filename in uploaded.keys():
    os.rename(filename, f"images/{filename}")
print("Images saved in the 'images/' directory.")


Images saved in the 'images/' directory.


In [3]:
uploaded = files.upload()

Saving data.json to data.json


In [4]:
!pip install transformers datasets torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [8]:
from datasets import load_dataset
import pandas as pd

# Load JSON or CSV dataset
dataset = load_dataset("json", data_files="/content/images/data.json")
print(dataset)


Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['image_id', 'image_path', 'caption'],
        num_rows: 4
    })
})


In [9]:
from transformers import AutoProcessor
from PIL import Image

processor = AutoProcessor.from_pretrained("microsoft/git-base")

def preprocess(sample):
    image = Image.open(sample['image_path']).convert('RGB')
    encoding = processor(images=image, text=sample['caption'], return_tensors="pt", padding=True)
    return encoding

processed_dataset = dataset.map(preprocess)


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [14]:
from transformers import AutoModelForVision2Seq, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split

# Split the dataset into training and evaluation sets
train_dataset, eval_dataset = train_test_split(
    processed_dataset['train'],
    test_size=0.2,
    random_state=42
)

# Load the model
model = AutoModelForVision2Seq.from_pretrained("microsoft/git-base")

# Define training arguments
training_args = TrainingArguments(
    output_dir="./fine_tuned_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",  # Optional: for logging
    logging_steps=10,      # Log every 10 steps
    learning_rate=2e-5,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,  # Optional: to handle learning rate smoothly
    weight_decay=0.01,  # Regularization
    save_total_limit=2,  # Keeps only the last 2 saved models
    report_to="none"  # Avoids logging to third-party services unless needed
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # Use the training split
    eval_dataset=eval_dataset,    # Provide the evaluation split
)

# Train the model
trainer.train()


TypeError: '<' not supported between instances of 'int' and 'ellipsis'

In [15]:
print(processed_dataset['train'][:5])